[WIP] https://realpython.com/pandas-project-gradebook/ 

### Loading the data

In [56]:
from pathlib import Path
import pandas as pd
import os
HERE = Path('__file__').parent
print(HERE)
DATA_FOLDER = HERE / "gradebook/data"
print(DATA_FOLDER)

In [58]:
roster = pd.read_csv(DATA_FOLDER / "roster.csv")
roster.head()

In [138]:
roster = pd.read_csv(
    DATA_FOLDER / "roster.csv",
    converters={"NetID": str.lower, "Email Address": str.lower},
    usecols=["Section", "Email Address", "NetID"],
    index_col="NetID",
)
roster.head()

,Email Address,Section
NetID,,
wxb12345,woody.barrera_jr@univ.edu,1
mxl12345,malaika.lambert@univ.edu,2
txj12345,traci.joyce@univ.edu,1
jgf12345,john.g.2.flower@univ.edu,3
smj00936,stacy.johnson@univ.edu,2


In [61]:
hw_exam_grades = pd.read_csv(DATA_FOLDER / "hw_exam_grades.csv")
hw_exam_grades.columns

In [139]:
hw_exam_grades = pd.read_csv(DATA_FOLDER / "hw_exam_grades.csv",
  converters={"SID": str.lower},
  usecols=lambda x: "Submission Time" not in x,
  index_col="SID")
hw_exam_grades.head()

,First Name,Last Name,Homework 1,Homework 1 - Max Points,Homework 2,Homework 2 - Max Points,Homework 3,Homework 3 - Max Points,Homework 4,Homework 4 - Max Points,...,Homework 9,Homework 9 - Max Points,Homework 10,Homework 10 - Max Points,Exam 1,Exam 1 - Max Points,Exam 2,Exam 2 - Max Points,Exam 3,Exam 3 - Max Points
SID,,,,,,,,,,,,,,,,,,,,,
axl60952,Aaron,Lester,68.0,80,74,80,77,80,89,100,...,42,60,41,60,79,100,63,100,68,100
amc28428,Adam,Cooper,80.0,80,78,80,78,80,87,100,...,45,60,53,60,65,100,63,100,90,100
axc64717,Alec,Curry,69.0,80,76,80,66,80,87,100,...,58,60,42,60,65,100,78,100,83,100
akr14831,Alexander,Rodriguez,50.0,80,54,80,74,80,75,100,...,38,60,47,60,97,100,97,100,81,100
axd11293,Amber,Daniels,54.0,80,57,80,77,80,95,100,...,46,60,59,60,68,100,95,100,88,100


In [140]:
quiz_grades = pd.DataFrame()
for file_path in DATA_FOLDER.glob("quiz_*_grades.csv"):    # https://docs.python.org/3/library/glob.html                                 
    quiz_name = "".join(file_path.stem.title().split("_")[:2]) # file_path.stem: Final path component without its suffix
    # print(file_path.stem, quiz_name)                    
    quiz = pd.read_csv(file_path, 
                      converters={"Email": str.lower},
                      index_col=["Email"],
                      usecols=["Email", "Grade"]).rename(columns={"Grade": quiz_name})
    quiz_grades = pd.concat([quiz_grades, quiz], axis=1)
quiz_grades

,Quiz1,Quiz4,Quiz5,Quiz2,Quiz3
Email,,,,,
richard.bennett@univ.edu,10,8,10,6,9
timothy.parker@univ.edu,9,14,10,14,13
carol.reyes@univ.edu,5,14,6,15,8
brooke.powers@univ.edu,6,10,8,10,17
michael.taylor@univ.edu,5,12,5,15,13
...,...,...,...,...,...
jeffrey.perez@univ.edu,4,12,9,7,12
angela.dunlap@univ.edu,6,11,6,11,11
richard.elliott@univ.edu,6,11,12,13,17


### Merging DataFrames

In [136]:
# Merge DataFrame or named Series objects with a database-style join.
# Default join is inner i.e to use intersection of keys from both frames
data = pd.merge(
    roster,
    hw_exam_grades,
    left_index=True,       # Use the index from the left DataFrame as the join key(s)
    right_index=True,      # Use the index from the right DataFrame as the join key
)

In [137]:
data

,Email Address,Section,First Name,Last Name,Homework 1,Homework 1 - Max Points,Homework 2,Homework 2 - Max Points,Homework 3,Homework 3 - Max Points,...,Homework 9,Homework 9 - Max Points,Homework 10,Homework 10 - Max Points,Exam 1,Exam 1 - Max Points,Exam 2,Exam 2 - Max Points,Exam 3,Exam 3 - Max Points
wxb12345,woody.barrera_jr@univ.edu,1,Woody,Barrera,55.0,80,62,80,73,80,...,45,60,46,60,86,100,62,100,90,100
mxl12345,malaika.lambert@univ.edu,2,Malaika,Lambert,63.0,80,57,80,78,80,...,45,60,37,60,60,100,91,100,93,100
txj12345,traci.joyce@univ.edu,1,Traci,Joyce,NaN,80,77,80,58,80,...,59,60,53,60,100,100,84,100,64,100
jgf12345,john.g.2.flower@univ.edu,3,Gregg,Flower,69.0,80,52,80,64,80,...,42,60,40,60,72,100,83,100,77,100
smj00936,stacy.johnson@univ.edu,2,Stacy,Johnson,74.0,80,55,80,60,80,...,59,60,48,60,96,100,80,100,86,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmj37756,paul.johnson@univ.edu,3,Paul,Johnson,73.0,80,50,80,55,80,...,40,60,51,60,84,100,80,100,94,100
dsl24347,danielle.lee@univ.edu,3,Danielle,Lee,69.0,80,51,80,70,80,...,49,60,57,60,62,100,70,100,90,100
nxe44872,nicole.edwards@univ.edu,3,Nicole,Edwards,62.0,80,76,80,62,80,...,58,60,51,60,89,100,63,100,65,100
bxr62103,bailey.reyes@univ.edu,2,Bailey,Reyes,53.0,80,50,80,55,80,...,41,60,46,60,90,100,72,100,71,100


In [142]:
final_data = pd.merge(data, quiz_grades, left_on="Email Address", right_index=True)
final_data

,Email Address,Section,First Name,Last Name,Homework 1,Homework 1 - Max Points,Homework 2,Homework 2 - Max Points,Homework 3,Homework 3 - Max Points,...,Exam 1 - Max Points,Exam 2,Exam 2 - Max Points,Exam 3,Exam 3 - Max Points,Quiz1,Quiz4,Quiz5,Quiz2,Quiz3
wxb12345,woody.barrera_jr@univ.edu,1,Woody,Barrera,55.0,80,62,80,73,80,...,100,62,100,90,100,4,7,10,10,11
mxl12345,malaika.lambert@univ.edu,2,Malaika,Lambert,63.0,80,57,80,78,80,...,100,91,100,93,100,8,13,6,10,10
txj12345,traci.joyce@univ.edu,1,Traci,Joyce,NaN,80,77,80,58,80,...,100,84,100,64,100,8,9,4,6,14
jgf12345,john.g.2.flower@univ.edu,3,Gregg,Flower,69.0,80,52,80,64,80,...,100,83,100,77,100,8,13,5,8,8
smj00936,stacy.johnson@univ.edu,2,Stacy,Johnson,74.0,80,55,80,60,80,...,100,80,100,86,100,6,7,7,14,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmj37756,paul.johnson@univ.edu,3,Paul,Johnson,73.0,80,50,80,55,80,...,100,80,100,94,100,10,11,10,14,9
dsl24347,danielle.lee@univ.edu,3,Danielle,Lee,69.0,80,51,80,70,80,...,100,70,100,90,100,7,5,7,14,10
nxe44872,nicole.edwards@univ.edu,3,Nicole,Edwards,62.0,80,76,80,62,80,...,100,63,100,65,100,10,12,10,12,9
bxr62103,bailey.reyes@univ.edu,2,Bailey,Reyes,53.0,80,50,80,55,80,...,100,72,100,71,100,10,6,7,15,17


In [144]:
final_data = final_data.fillna(0)
final_data

,Email Address,Section,First Name,Last Name,Homework 1,Homework 1 - Max Points,Homework 2,Homework 2 - Max Points,Homework 3,Homework 3 - Max Points,...,Exam 1 - Max Points,Exam 2,Exam 2 - Max Points,Exam 3,Exam 3 - Max Points,Quiz1,Quiz4,Quiz5,Quiz2,Quiz3
wxb12345,woody.barrera_jr@univ.edu,1,Woody,Barrera,55.0,80,62,80,73,80,...,100,62,100,90,100,4,7,10,10,11
mxl12345,malaika.lambert@univ.edu,2,Malaika,Lambert,63.0,80,57,80,78,80,...,100,91,100,93,100,8,13,6,10,10
txj12345,traci.joyce@univ.edu,1,Traci,Joyce,0.0,80,77,80,58,80,...,100,84,100,64,100,8,9,4,6,14
jgf12345,john.g.2.flower@univ.edu,3,Gregg,Flower,69.0,80,52,80,64,80,...,100,83,100,77,100,8,13,5,8,8
smj00936,stacy.johnson@univ.edu,2,Stacy,Johnson,74.0,80,55,80,60,80,...,100,80,100,86,100,6,7,7,14,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmj37756,paul.johnson@univ.edu,3,Paul,Johnson,73.0,80,50,80,55,80,...,100,80,100,94,100,10,11,10,14,9
dsl24347,danielle.lee@univ.edu,3,Danielle,Lee,69.0,80,51,80,70,80,...,100,70,100,90,100,7,5,7,14,10
nxe44872,nicole.edwards@univ.edu,3,Nicole,Edwards,62.0,80,76,80,62,80,...,100,63,100,65,100,10,12,10,12,9
bxr62103,bailey.reyes@univ.edu,2,Bailey,Reyes,53.0,80,50,80,55,80,...,100,72,100,71,100,10,6,7,15,17


### Calculating Grades